In [1]:
import numpy as np
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from glob import glob
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 수질정보의 년도는 2,5,8,11 로 구성되어 있음
# 부유물수거량 데이터의 작업일은 연속값
# 수질정보의 년도에 데이터를 넣기 위해 연속값을 카테고리시키는 작업
df = pd.read_csv('../../data/team/해양환경측정망_합본_2.csv')
df1 = pd.read_csv('../../data/team/classify_area.csv')

df2 = df1[df1['작업일'] > 20141131]
df2['년월'] = ''
for j in range(0,39999,10000):
    for i in range(0,1119,300) :
        # 12~2월 데이터를 넣기 때문에 if문으로 년도가 바뀌는 것을 구분해줌
        if i == 0:
            a = df2[(df2['작업일']< 20150331+i+j)&(df2['작업일'] >= 20141201+j)]
        else:
            a = df2[(df2['작업일']< 20150231+i+j)&(df2['작업일'] >= 20149901+i+j)]
        for z in a.index:
            df2.loc[z,'년월'] = round((20150201+i+j)/100)
            
df2['작업일'] = df2['년월']
df2 = df2.drop('년월', axis=1)
mean = df2.groupby(['작업일','지역'])['수거량'].mean()
mean1 = df2.groupby(['작업일','생태구역'])['수거량'].mean()

In [7]:
df3 = pd.DataFrame(data=[[1,2,3]],columns=['년월','지역','수거량'])
a = 0
for i in mean.index:
    # 멀티인덱스이기 때문에 하나씩 호출, 단일인덱스 변환
    df3.loc[a,'년월'] = i[0]
    df3.loc[a,'지역'] = i[1]
    df3.loc[a,'수거량'] = mean[i]
    a +=1
    
append_l = pd.DataFrame(data=[[1,2,3]],columns=['년월','생태구역','수거량'])
a= 0
for i in mean1.index:
    append_l.loc[a,'년월'] = i[0]
    append_l.loc[a,'생태구역'] = i[1]
    append_l.loc[a,'수거량'] = mean1[i]
    a +=1

In [9]:
ym = df[['년','월']].astype(float)
ym1 = ym['년']*100
ym2 = ym1+ym['월']

df['년'] = ym2
df = df.drop(['월'],axis=1)
# 부유물데이터는 19년도가 없기 때문에 제거함
df.rename(columns={'년':'년월'}, inplace=True)
df = df[df['년월'] <201901]

In [11]:
df['수거량'] = ''
index_l = df.index
area_1 = df3['지역'].unique()
area_2 = append_l['생태구역'].unique()

# 전체데이터의 인덱스를 통해서 년도, 지역에 맞는 데이터를 넣어줌
for i in index_l:
    for j in area_2:
        if df.loc[i,'생태구역'] == j:
            y = df.loc[i,'년월']
            trash = append_l[(append_l['년월'] == y)&(append_l['생태구역'] == j)]['수거량'].iloc[0]
            df.loc[i,'수거량'] = trash

for i in index_l:
    for j in area_1:
        if df.loc[i,'구군'] == j:
            y = df.loc[i,'년월']
            trash = df3[(df3['년월'] == y)&(df3['지역'] == j)]['수거량'].iloc[0]
            df.loc[i,'수거량'] = trash
        

In [12]:
df.to_csv('../../data/team/수질+수거량.csv', index=False, encoding='utf-8')
df.to_csv('../../data/team/수질+수거량_R.csv', index=False, encoding='ANSI')